In [0]:
pip install easyocr

## Import libraries

In [0]:
import easyocr as ocr
from PIL import Image
import numpy as np
import requests
from io import BytesIO

### Load model

In [0]:
# load in ocr model
def load_model():
    reader = ocr.Reader(['en'], model_storage_directory='.')
    return reader

reader = load_model()

### Function to format text result

In [0]:
def extract_text(input_image):
    result = reader.readtext(np.array(input_image))

    # format text
    result_text = []

    for text in result:
        result_text.append(text[1])
    
    img_text = ' '.join(map(str,result_text))
    
    return img_text

### Load image from URL

In [0]:
# request URL response
url = "https://www.forbesindia.com/media/images/2023/May/img_207565_mayaangelou.jpg"
response = requests.get(url)

# open image content
img = Image.open(BytesIO(response.content))

In [0]:
# display image
img

In [0]:
extract_text(img)

### Load image from file

In [0]:
# img file path
img_file = "assets/quote.jpeg"

# load image
img = Image.open(img_file)

In [0]:
# display image
img

In [0]:
extract_text(img)

In [0]:
results = reader.readtext(np.array(img))

### filter results with margin

In [0]:
# get image dimensions
image_width, image_height = img.size

In [0]:
percentage = 0.80  # Percentage of the center region (e.g., 75%)

# Calculate the margin based on the image size and percentage
margin_x = int(image_width * (1 - percentage) / 2)
margin_y = int(image_height * (1 - percentage) / 2)

filtered_results = []
for result in results:
    bbox = result[0]
    bbox_width = bbox[2][0] - bbox[0][0]
    bbox_height = bbox[2][1] - bbox[0][1]
    center_x = (bbox[0][0] + bbox[2][0]) / 2
    center_y = (bbox[0][1] + bbox[2][1]) / 2
    
    if (
        center_x > margin_x and center_x < (image_width - margin_x) and
        center_y > margin_y and center_y < (image_height - margin_y) and
        bbox_width > margin_x and bbox_height > margin_y
    ):
        filtered_results.append(result)


In [0]:
result_text = []
for text in filtered_results:
    result_text.append(text[1])

In [0]:
result_text

margin = 350  # Margin in pixels

filtered_results = []
for result in results:
    # Extract the coordinates of the bounding box
    bbox = result[0]
    
    # Check if the bounding box is within the margin
    if bbox[0][0] > margin and bbox[0][1] > margin and bbox[2][0] < (image_width - margin) and bbox[2][1] < (image_height - margin):
        filtered_results.append(result)